In [10]:
#Import everything needed for matplotlib
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [11]:
#Import of libraries to be used
import numpy as np
import pandas as pd
import datetime as dt

In [12]:
 # Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [13]:
#Create the engine
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [15]:
#Create the base and automap the classes
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['measurement', 'station']

In [16]:
#Now to assign the classes to 
Station = Base.classes.station
Measurement = Base.classes.measurement

In [17]:
#Create a session
session = Session(engine)

In [44]:
#first_row_station = session.query(Station).first()
#first_row_station.__dict__

In [83]:
#first_row_measurement = session.query(Measurement).first()
#first_row_measurement.__dict__

In [102]:
#For some reason the 
top_date = session.query(func.max(func.strftime("%Y-%m-%d", Measurement.date)))
top_date_date = dt.datetime.strptime(top_date[0][0], "%Y-%m-%d")
year_ago = top_date_date - dt.timedelta(weeks=52.2)
#type(top_date[0][0])

In [103]:
qry = session.query(Measurement.date, Measurement.prcp).\
                        filter(Measurement.date >= year_ago)
test_df = pd.read_sql(qry.statement, session.bind)
test_df.set_index('date')

,prcp
date,
2016-08-23,0.00
2016-08-24,0.08
2016-08-25,0.08
2016-08-26,0.00
2016-08-27,0.00
...,...
2017-08-19,0.09
2017-08-20,NaN
2017-08-21,0.56
